In [1]:
import pandas as pd
import re
import requests

from bs4 import BeautifulSoup 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException


import time

In [2]:
productDF = pd.read_csv('./productInfo/product(1).csv', index_col=0)
productDF.head()

,product_idx,video_id,product_id_inshop,product_url,shoppingmall,short_url,product_name,original_price,current_price,discount_rate,brand_name,brand_description,brand_url_inshop,product_fullinfo
0,24,fflimV2pIL8,6252042,https://gift.kakao.com/product/6252042,카카오 선물하기,https://gift.kakao.com/product/6252042,[리본쇼핑백&헤어스크런치 증정] '팩과 클렌징을 한 번에' 휩드 비건 팩클렌저 1...,22700,22700,0,휩드,NaN,https://gift.kakao.com/brand/14844,"{'product_name': "" [리본쇼핑백&헤어스크런치 증정] '팩과 클렌징을 ..."
1,28,ONFJdkVFSzY,6784177,https://gift.kakao.com/product/6784177,카카오 선물하기,https://gift.kakao.com/product/6784177,월레스와 그로밋 일회용 필름 카메라 - 화려한 외출,27900,27900,0,콜리,NaN,https://gift.kakao.com/brand/14472,{'product_name': ' 월레스와 그로밋 일회용 필름 카메라 - 화려한 외...
2,29,ONFJdkVFSzY,5627132,https://gift.kakao.com/product/5627132,카카오 선물하기,https://gift.kakao.com/product/5627132,[단독+포장] 바디오일 & 샌달우드 괄사&에센셜오일 세트 / 3종 택1,29000,29000,0,톤28,NaN,https://gift.kakao.com/brand/10336,{'product_name': ' [단독+포장] 바디오일 & 샌달우드 괄사&에센셜오...
3,30,ONFJdkVFSzY,6218926,https://gift.kakao.com/product/6218926,카카오 선물하기,https://gift.kakao.com/product/6218926,N1 DE CHANEL 레드 까멜리아 립 앤 치크 밤,63000,63000,0,샤넬,NaN,https://gift.kakao.com/brand/10813,{'product_name': ' N1 DE CHANEL 레드 까멜리아 립 앤 치크...
4,31,kMHj6Zv4bW8,7695898,https://gift.kakao.com/product/7695898,카카오 선물하기,https://gift.kakao.com/product/7695898,"[단독/리본포장] ""푸른 텍스처+애프터배스 향"" 블루 웨이브 바디 오일 120ML",33000,33000,0,센녹,NaN,https://gift.kakao.com/brand/15661,"{'product_name': ' [단독/리본포장] ""푸른 텍스처+애프터배스 향"" ..."


In [3]:
def remove_non_numeric(input_string):
    # 숫자가 아닌 모든 문자를 찾는 정규 표현식 패턴
    pattern = r'[^0-9]'
    # 숫자가 아닌 모든 문자를 빈 문자열로 대체
    cleaned_string = re.sub(pattern, '', input_string)
    return int(cleaned_string)

In [30]:
def get_kko_review(url):
    # Selenium WebDriver 설정
    driver = webdriver.Chrome() 
    driver.get(url + '?tab=review&sortProperty=LATEST') 

    # 필요한 값이 로드될 때 까지 기다림
    try : 
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'div.wrap_group.wrap_reviewcard'))
        )
    except TimeoutException as e:
        driver.quit()
        return None


    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 값 가져오기
    try:
        no_review = soup.select_one('div.wrap_group.wrap_noreview')
        if no_review:
            return 0, None
    except:
        pass
    
    
    review_count = remove_non_numeric(soup.select_one('div.wrap_group.wrap_reviewcard').select_one('h4.tit_group').get_text())
    review_wrapper = soup.select_one('div.wrap_group.wrap_reviewcard').select_one('ul.list_review')
    # except: 
    #     return -1, None # 판매 중지 혹은 품절 

    # 더보기 미리 클릭
    if (review_count - 1) // 20 > 0:
        if (review_count - 1) // 20 > 5 : max_click = 5
        else: max_click = (review_count - 1) // 20
        
        for press_count in range(max_click):
            try:
                driver.find_element(By.XPATH, '//*[@id="tabPanel_review"]/div/div[2]/button').click()
                time.sleep(2)
            except:
                break
    # 리뷰 수집
    review_lst = []
    for review in review_wrapper.select('app-view-review-item'):
        star_rate = review.select_one('em.ico_detail').get_text()
        created_at = review.select('span.txt_reviewinfo')[-1].get_text()
        review_text = review.select_one('p.txt_review').get_text()
        # print(star_rate,created_at,review_text)
        review_lst.append(dict(star_rate=star_rate,created_at=created_at,review_text=review_text))
        time.sleep(2)

    # 불러온 값이 None 일 때의 대비 아직 미완

    # 브라우저 닫기
    driver.quit()

    return review_count, review_lst

In [5]:
def get_29cm_review(url):
    # Selenium WebDriver 설정
    driver = webdriver.Chrome() 
    driver.get(url) 
    driver.find_element(By.XPATH, '//*[@id="__next"]/div[5]/div[2]/div[2]/div[1]/div/div[2]/button').click()

    # 필요한 값이 로드될 때 까지 기다림
    try : 
        WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'span.css-1f0l480.ex98du3'))#'div.css-70qvj9.ex98du4'))# 'section.e6fth966.css-1w043rb.ex98du0'))
        )
    except TimeoutException as e:
        driver.quit()

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 값 가져오기
    review_count = remove_non_numeric(soup.select_one('span.css-1f0l480.ex98du3').get_text())
    review_pages = int(soup.select_one('div.e13i1jpn4.css-1oq0g9s.ej7aofc0').select_one('ul.css-16vmvyd.ej7aofc1').select('li')[-1].get_text())
    if review_pages > 5:
        review_pages = 5

    review_lst = []
    for i in range(1, review_pages+1):
        driver.find_element(By.XPATH, f'//*[@id="__next"]/div[7]/section[5]/div[3]/div/ul/li[{i}]/button').click()

        review_wrapper = soup.select_one('section.e6fth966.css-1w043rb.ex98du0').select_one('ul.css-0.e13i1jpn1')

        for review in review_wrapper.select('li'):
            star_wrapper = review.select_one('div.css-18biwo.e8ryq2d0').select('i.css-9nop8.e8ryq2d1')
            star_rate = 0
            for star in star_wrapper:
                if star.select_one('i.css-jcf0hl.e8ryq2d2') : star_rate += 1
            created_at = review.select_one('span.css-1riowxi.eji1c1x6').get_text()
            review_text = review.select_one('p.css-1yblk9b.eji1c1x8').get_text()
            review_lst.append(dict(star_rate=star_rate,created_at=created_at,review_text=review_text))
            time.sleep(3)

    # 불러온 값이 None 일 때의 대비 아직 미완

    # 브라우저 닫기
    driver.quit()

    return review_count, review_lst


In [13]:
review_num = []
review_sample = []
for idx, url in enumerate(productDF['short_url'][:3]):
    if productDF['shoppingmall'][idx] == "카카오 선물하기":
        try: 
            temp_renum, temp_resample =  get_kko_review(url)
            review_num.append(temp_renum)
            review_sample.append(temp_resample)
        except:
            print(f"excepted : {idx}")
            review_num.append(None)
            review_sample.append(None)
    else:
        try: 
            temp_renum, temp_resample =  get_29cm_review(url)
            review_num.append(temp_renum)
            review_sample.append(temp_resample)
        except:
            print(f"excepted : {idx}")
            review_num.append(None)
            review_sample.append(None)

In [ ]:
productDF['review_num'] = review_num
productDF['review_sample'] = review_sample
productDF.to_csv('./productInfo/with_reviews.csv')

#### ERROR 난 url 확인
Case Study
- excepted :  3 -> 판매 종료 (다른 상품 코드로 같은 제품 있음)
- excepted :  7 -> 선물 후기가 없음 ```해결``` / ```채워넣기 완료```
- excepted : 11 -> 품절 & 리뷰 없음 
- excepted : 12 -> 판매 종료 (판매 종료 팝업이 뜨면서 더보기가 안눌리는 현상)
- excepted : 19 -> 판매 종료
- excepted : 27 -> 판매 종료
- excepted : 28 -> 판매 종료
- excepted : 31 -> 품절
- excepted : 34 -> 선물 후기가 없음 ```해결```, ```채워넣기 완료```
- excepted : 37 -> 판매 종료
- excepted : 44 -> 판매 종료
- excepted : 59 -> 다시 돌렸을 때 문제 발생 X ```해결?``` / ```채워넣기 완료```
- excepted : 60 -> 다시 돌렸을 때 문제 발생 X ```해결?``` / ```채워넣기 완료```
- excepted : 61 -> 다시 돌렸을 때 문제 발생 X ```해결?``` / ```채워넣기 완료```
- excepted : 66 -> 다시 돌렸을 때 문제 발생 X ```해결?``` / ```채워넣기 완료```

결론 
- 카톡 선물 : 리뷰가 없는 경우 예외 처리, 판매 종료/품절의 경우 예외 처리
- 29cm    : time out error.. -> time out 발생 시에는 기록해뒀다가 다음에 몇 번 더 시도하는 방식으로 해결

### 추가된 Task
1. Product crawling
    - 판매 중지 된 상품, 품절된 상품 처리 코드 수정
    - 카톡 선물 : 리뷰 태그
    - 29cm    : 선물 리뷰인지 체크, 
2. Review crawling 
    - 29cm    : 리뷰 없을 때 처리
    - Crawling Date 추가하기